In [8]:
!pip install -q -U transformers peft accelerate optimum bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.9 MB/s eta 0:00:00


In [2]:
!pip install -q -U huggingface_hub

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, TextStreamer

In [2]:
model=AutoModelForCausalLM.from_pretrained('jphme/em_german_7b_v01', low_cpu_mem_usage=True, load_in_8bit=True)
tokenizer=AutoTokenizer.from_pretrained('jphme/em_german_7b_v01')
tokenizer.pad_token_id=tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
generation_config=GenerationConfig(max_new_tokens=500,
                                    temperature=0.4,
                                    top_p=0.95,
                                    top_k=40,
                                    repetition_penalty=1.2,
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    do_sample=True,
                                    use_cache=True,
                                    output_attentions=False,
                                    output_hidden_states=False,
                                    output_scores=False,
                                    remove_invalid_values=True
                                    )
streamer = TextStreamer(tokenizer)

In [1]:
def ask_model(instruction, system='Du bist ein hilfreicher Assistent.'):
    prompt=f"{system} USER: {instruction} ASSISTANT:"
    input_tokens=tokenizer(prompt, return_tensors="pt").to(model.device)
    output_tokens=model.generate(**input_tokens,  generation_config=generation_config, streamer=streamer)[0]
    answer=tokenizer.decode(output_tokens, skip_special_tokens=True)
    return answer

In [7]:
answer=ask_model('Wenn du 2024 als Bundeskanzler kandidieren würdest, was wäre dein Wahlprogramm?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Du bist ein hilfreicher Assistent. USER: Wenn du 2024 als Bundeskanzler kandidieren würdest, was wäre dein Wahlprogramm? ASSISTANT: Als Kandidat für das Amt des Bundespräsidenten im Jahr 2024 möchte ich Ihnen meine politischen Vorschläge darlegen und wie sie sich auf verschiedene Themenfelder auswirken könnten.

1. Wirtschaftspolitik: Die Förderung der nachhaltigen Entwicklung durch den Ausbau erneuerbarer Energien, die Verringerung der CO2-Emissionen bis zum Jahr 2035 um mindestens 65% gegenüber dem Niveau von 1990 sowie eine stärkere Investition in Forschung und Entwicklung zur Verbesserung unseres technologischen Fortschritts sind wichtige Ziele. Darüber hinaus strebe ich an, einen faireren Arbeitsmarkt zu schaffen, indem wir Maßnahmen gegen soziale Ungleichheit ergreifen, um die Einkommensverteilung zugunsten eher unterversorgter Bevölkerungsgruppen auszugleichen.

2. Sozialpolitik: Um die Lebensqualität aller Bürgerinnen und Bürger zu verbessern, ist es notwendig, den Zugang z

# Example for Factual/Retrieval with less Hallucinations

Please use the Format and system message below for Factual retrieval. The model is trained to only answer from provided context and not hallucinate any answers or additional information.

In [8]:
retrieval_system="Du bist ein hilfreicher Assistent. Für die folgende Aufgabe stehen dir zwischen den tags BEGININPUT und ENDINPUT mehrere Quellen zur Verfügung. Metadaten zu den einzelnen Quellen wie Autor, URL o.ä. sind zwischen BEGINCONTEXT und ENDCONTEXT zu finden, danach folgt der Text der Quelle. Die eigentliche Aufgabe oder Frage ist zwischen BEGININSTRUCTION und ENDINCSTRUCTION zu finden. Beantworte diese wortwörtlich mit einem Zitat aus den Quellen. Sollten diese keine Antwort enthalten, antworte, dass auf Basis der gegebenen Informationen keine Antwort möglich ist!"
retrieval_question="""\
BEGININPUT
BEGINCONTEXT
Url: https://www.jph.me
ENDCONTEXT
Das Wetter in Düsseldorf wird heute schön und sonnig!
ENDINPUT
BEGININSTRUCTION Was ist 1+1? ENDINSTRUCTION"""
ask_model(retrieval_question, system=retrieval_system)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Du bist ein hilfreicher Assistent. Für die folgende Aufgabe stehen dir zwischen den tags BEGININPUT und ENDINPUT mehrere Quellen zur Verfügung. Metadaten zu den einzelnen Quellen wie Autor, URL o.ä. sind zwischen BEGINCONTEXT und ENDCONTEXT zu finden, danach folgt der Text der Quelle. Die eigentliche Aufgabe oder Frage ist zwischen BEGININSTRUCTION und ENDINCSTRUCTION zu finden. Beantworte diese wortwörtlich mit einem Zitat aus den Quellen. Sollten diese keine Antwort enthalten, antworte, dass auf Basis der gegebenen Informationen keine Antwort möglich ist! USER: BEGININPUT
BEGINCONTEXT
Url: https://www.jph.me
ENDCONTEXT 
Das Wetter in Düsseldorf wird heute schön und sonnig!
ENDINPUT
BEGININSTRUCTION Was ist 1+1? ENDINSTRUCTION ASSISTANT: Mit den gegebenen Informationen ist diese Frage nicht zu beantworten.</s>


'Du bist ein hilfreicher Assistent. Für die folgende Aufgabe stehen dir zwischen den tags BEGININPUT und ENDINPUT mehrere Quellen zur Verfügung. Metadaten zu den einzelnen Quellen wie Autor, URL o.ä. sind zwischen BEGINCONTEXT und ENDCONTEXT zu finden, danach folgt der Text der Quelle. Die eigentliche Aufgabe oder Frage ist zwischen BEGININSTRUCTION und ENDINCSTRUCTION zu finden. Beantworte diese wortwörtlich mit einem Zitat aus den Quellen. Sollten diese keine Antwort enthalten, antworte, dass auf Basis der gegebenen Informationen keine Antwort möglich ist! USER: BEGININPUT\nBEGINCONTEXT\nUrl: https://www.jph.me\nENDCONTEXT \nDas Wetter in Düsseldorf wird heute schön und sonnig!\nENDINPUT\nBEGININSTRUCTION Was ist 1+1? ENDINSTRUCTION ASSISTANT: Mit den gegebenen Informationen ist diese Frage nicht zu beantworten.'

In [11]:
retrieval_system="Du bist ein hilfreicher Assistent. Für die folgende Aufgabe stehen dir zwischen den tags BEGININPUT und ENDINPUT mehrere Quellen zur Verfügung. Metadaten zu den einzelnen Quellen wie Autor, URL o.ä. sind zwischen BEGINCONTEXT und ENDCONTEXT zu finden, danach folgt der Text der Quelle. Die eigentliche Aufgabe oder Frage ist zwischen BEGININSTRUCTION und ENDINCSTRUCTION zu finden. Beantworte diese wortwörtlich mit einem Zitat aus den Quellen. Sollten diese keine Antwort enthalten, antworte, dass auf Basis der gegebenen Informationen keine Antwort möglich ist!"
retrieval_question="""\
BEGININPUT
BEGINCONTEXT
Url: https://www.jph.me
ENDCONTEXT
Das Wetter in Düsseldorf wird heute schön und sonnig!
ENDINPUT
BEGININSTRUCTION Wie wird das Wetter heute in Düsseldorf? ENDINSTRUCTION"""
ask_model(retrieval_question, system=retrieval_system)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Du bist ein hilfreicher Assistent. Für die folgende Aufgabe stehen dir zwischen den tags BEGININPUT und ENDINPUT mehrere Quellen zur Verfügung. Metadaten zu den einzelnen Quellen wie Autor, URL o.ä. sind zwischen BEGINCONTEXT und ENDCONTEXT zu finden, danach folgt der Text der Quelle. Die eigentliche Aufgabe oder Frage ist zwischen BEGININSTRUCTION und ENDINCSTRUCTION zu finden. Beantworte diese wortwörtlich mit einem Zitat aus den Quellen. Sollten diese keine Antwort enthalten, antworte, dass auf Basis der gegebenen Informationen keine Antwort möglich ist! USER: BEGININPUT
BEGINCONTEXT
Url: https://www.jph.me
ENDCONTEXT 
Das Wetter in Düsseldorf wird heute schön und sonnig!
ENDINPUT
BEGININSTRUCTION Wie wird das Wetter heute in Düsseldorf? ENDINSTRUCTION ASSISTANT: Das Wetter in Düsseldorf wird heute schön und sonnig!</s>


'Du bist ein hilfreicher Assistent. Für die folgende Aufgabe stehen dir zwischen den tags BEGININPUT und ENDINPUT mehrere Quellen zur Verfügung. Metadaten zu den einzelnen Quellen wie Autor, URL o.ä. sind zwischen BEGINCONTEXT und ENDCONTEXT zu finden, danach folgt der Text der Quelle. Die eigentliche Aufgabe oder Frage ist zwischen BEGININSTRUCTION und ENDINCSTRUCTION zu finden. Beantworte diese wortwörtlich mit einem Zitat aus den Quellen. Sollten diese keine Antwort enthalten, antworte, dass auf Basis der gegebenen Informationen keine Antwort möglich ist! USER: BEGININPUT\nBEGINCONTEXT\nUrl: https://www.jph.me\nENDCONTEXT \nDas Wetter in Düsseldorf wird heute schön und sonnig!\nENDINPUT\nBEGININSTRUCTION Wie wird das Wetter heute in Düsseldorf? ENDINSTRUCTION ASSISTANT: Das Wetter in Düsseldorf wird heute schön und sonnig!'